<a href="https://colab.research.google.com/github/AndrewSalim048/Andrew-Salim_final_project/blob/main/copy-Project_tahap_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Analisis Big Data (A)


---


# Project Tahap 1 

Anggota:

1.   Belgis Berliana Putri (205150201111065)
2.   Hilyatul Jannah (205150201111068)
3.   Roid Muhammad Syafiq (205150201111027)
4.   a
5.   Andrew Salim (205150207111048)



## Load Data

In [ ]:
! wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings/Public/data/news_category_train.csv
! wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings/Public/data/news_category_test.csv

Import pandas dan simpan news_category_test.csv pada df_train. Pada fungsi read_csv tambahkan error_bad_lines=False. Alasan penambahan parameter tersebut karena terpadat error ketika mengisikan data pada baris 5269 dan 14029 dari file csv tersebut.

## Penggunaan BERT dengan Spark Map Reduce

In [ ]:
!wget https://setup.johnsnowlabs.com/colab.sh -O - | bash /dev/stdin -p 3.3.0 -s 4.3.2 -g

--2023-05-03 09:23:19--  https://setup.johnsnowlabs.com/colab.sh
Resolving setup.johnsnowlabs.com (setup.johnsnowlabs.com)... 51.158.130.125
Connecting to setup.johnsnowlabs.com (setup.johnsnowlabs.com)|51.158.130.125|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp/master/scripts/colab_setup.sh [following]
--2023-05-03 09:23:19--  https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp/master/scripts/colab_setup.sh
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1191 (1.2K) [text/plain]
Saving to: ‘STDOUT’

-                   100%[===================>]   1.16K  --.-KB/s    in 0s      

2023-05-03 09:23:19 (53.4 MB/s) - written to stdout [1191/1191]

Installi

In [ ]:
import sparknlp

from sparknlp.base import *
from sparknlp.annotator import *
from pyspark.ml import Pipeline
import pandas as pd
import os

spark = sparknlp.start(gpu = True)# for GPU training >> sparknlp.start(gpu = True)

print("Spark NLP version", sparknlp.version())
print("Apache Spark version:", spark.version)

spark

Spark NLP version 4.3.2
Apache Spark version: 3.3.0


In [ ]:
trainDataset = spark.read \
      .option("header", True) \
      .csv("news_category_train.csv")

trainDataset.show(truncate=50)

+--------+--------------------------------------------------+
|category|                                       description|
+--------+--------------------------------------------------+
|Business| Short sellers, Wall Street's dwindling band of...|
|Business| Private investment firm Carlyle Group, which h...|
|Business| Soaring crude prices plus worries about the ec...|
|Business| Authorities have halted oil export flows from ...|
|Business| Tearaway world oil prices, toppling records an...|
|Business| Stocks ended slightly higher on Friday but sta...|
|Business| Assets of the nation's retail money market mut...|
|Business| Retail sales bounced back a bit in July, and n...|
|Business|" After earning a PH.D. in Sociology, Danny Baz...|
|Business| Short sellers, Wall Street's dwindling  band o...|
|Business| Soaring crude prices plus worries  about the e...|
|Business| OPEC can do nothing to douse scorching  oil pr...|
|Business| Non OPEC oil exporters should consider  increa...|
|Busines

In [ ]:
# hapus baris 5269 dan 14029

In [ ]:
trainDataset.count()

120000

In [ ]:
from pyspark.sql.functions import col

trainDataset.groupBy("category") \
    .count() \
    .orderBy(col("count").desc()) \
    .show()

+--------+-----+
|category|count|
+--------+-----+
|   World|30000|
|Sci/Tech|30000|
|  Sports|30000|
|Business|30000|
+--------+-----+



In [ ]:
from pyspark.sql.functions import col

testDataset = spark.read \
      .option("header", True) \
      .csv("news_category_test.csv")


testDataset.groupBy("category") \
      .count() \
      .orderBy(col("count").desc()) \
      .show()

+--------+-----+
|category|count|
+--------+-----+
|   World| 1900|
|Sci/Tech| 1900|
|  Sports| 1900|
|Business| 1900|
+--------+-----+



Buat model

In [ ]:
document_assembler = DocumentAssembler() \
    .setInputCol("description") \
    .setOutputCol("document")

tokenizer = Tokenizer() \
    .setInputCols(["document"]) \
    .setOutputCol("token")

bert_embeddings = BertEmbeddings().pretrained(name='small_bert_L4_256', lang='en') \
    .setInputCols(["document",'token'])\
    .setOutputCol("embeddings")

embeddingsSentence = SentenceEmbeddings() \
    .setInputCols(["document", "embeddings"]) \
    .setOutputCol("sentence_embeddings") \
    .setPoolingStrategy("AVERAGE")

classsifierdl = ClassifierDLApproach()\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("class")\
    .setLabelColumn("category")\
    .setMaxEpochs(10)\
    .setLr(0.001)\
    .setBatchSize(8)\
    .setEnableOutputLogs(True)
    #.setOutputLogsPath('logs')

bert_clf_pipeline = Pipeline(stages=[document_assembler,
                                     tokenizer,
                                     bert_embeddings,
                                     embeddingsSentence,
                                     classsifierdl])

small_bert_L4_256 download started this may take some time.
Approximate size to download 40.5 MB
[OK!]


In [ ]:
# remove the existing logs
# Jika sebelumnya pernah melatih model, hapus annotator_logs nya

! rm -r /root/annotator_logs

rm: cannot remove '/root/annotator_logs': No such file or directory


In [ ]:
%%time

bert_clf_pipelineModel = bert_clf_pipeline.fit(trainDataset)

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/usr/local/lib/python3.10/dist-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/lib/python3.10/socket.py", line 705, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: ignored

In [ ]:
log_files = os.listdir("/root/annotator_logs")
log_files

FileNotFoundError: ignored

In [ ]:
log_file_name = os.listdir("/root/annotator_logs")[0]

# Lihat proses pelatihan model melalui annotator_logs
with open("/root/annotator_logs/"+log_file_name, "r") as log_file :
    print(log_file.read())

In [ ]:
# We are going to use sklearn to evalute the results on test dataset
# Pakai sklearn untuk mengevaluasi metriks-nya
from sklearn.metrics import classification_report

preds = bert_clf_pipelineModel.transform(testDataset)

preds_df = preds.select('category','description',"class.result").toPandas()

preds_df['result'] = preds_df['result'].apply(lambda x : x[0])

print (classification_report(preds_df['category'], preds_df['result']))


## Penggunaan BERT tanpa Spark Map Reduce

In [ ]:
import pandas as pd

df_train = pd.read_csv("news_category_train.csv", error_bad_lines=False)
# ada error di sekitar baris 5269 dan 14029 sebab dianggap ada lebih dari 2 atribut

<ipython-input-13-50e2fe354cb4>:3: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df_train = pd.read_csv("news_category_train.csv", error_bad_lines=False)
Skipping line 5269: expected 2 fields, saw 9
Skipping line 14029: expected 2 fields, saw 3



Lihat 20 baris pertama

In [ ]:
df_train.head(20)

,category,description
0,Business,"Short sellers, Wall Street's dwindling band o..."
1,Business,"Private investment firm Carlyle Group, which ..."
2,Business,Soaring crude prices plus worries about the e...
3,Business,Authorities have halted oil export flows from...
4,Business,"Tearaway world oil prices, toppling records a..."
5,Business,Stocks ended slightly higher on Friday but st...
6,Business,Assets of the nation's retail money market mu...
7,Business,"Retail sales bounced back a bit in July, and ..."
8,Business,"After earning a PH.D. in Sociology, Danny Baz..."
9,Business,"Short sellers, Wall Street's dwindling band ..."


Lihat informasi yang ada pada df_train

In [ ]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119998 entries, 0 to 119997
Data columns (total 2 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   category     119998 non-null  object
 1   description  119998 non-null  object
dtypes: object(2)
memory usage: 1.8+ MB


Lihat total banyak macam-macam nilai pada kolom category

In [ ]:
df_train["category"].value_counts()

Sports      30000
World       30000
Business    29999
Sci/Tech    29999
Name: category, dtype: int64

Lakukan proses yang sama seperti sebelumnya pada file news_category_test.csv

In [ ]:
df_test = pd.read_csv("news_category_test.csv")

In [ ]:
df_test.head()

,category,description
0,Business,Unions representing workers at Turner Newall...
1,Sci/Tech,"TORONTO, Canada A second team of rocketeer..."
2,Sci/Tech,A company founded by a chemistry researcher a...
3,Sci/Tech,It's barely dawn when Mike Fitzpatrick starts...
4,Sci/Tech,Southern California's smog fighting agency we...


In [ ]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7600 entries, 0 to 7599
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   category     7600 non-null   object
 1   description  7600 non-null   object
dtypes: object(2)
memory usage: 118.9+ KB


In [ ]:
df_test["category"].value_counts()

Business    1900
Sci/Tech    1900
Sports      1900
World       1900
Name: category, dtype: int64

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub

# Pakai pre-trained BERT model
module_url = "https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-256_A-4/2"
bert_layer = hub.KerasLayer(module_url, trainable=True)

In [ ]:
# Tokenize data menggunakan pre-trained BERT tokenizer
tokenizer = tf.keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(df_train["description"])
tokenized_data = tokenizer.texts_to_sequences(df_train["description"])

In [ ]:
# Padding tokenized data
max_seq_length = 128
padded_data = tf.keras.preprocessing.sequence.pad_sequences(tokenized_data, maxlen=max_seq_length, padding="post")

In [ ]:
###
print(tf.__version__)
print(hub.__version__)

padded_data = tf.cast(padded_data, tf.int32)

2.12.0
0.13.0


In [ ]:
###
!pip install tensorflow-text

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 9.9 MB/s eta 0:00:00


In [ ]:
# Eksekusi pemrosesan data menggunakan model BERT
"""
inputs = bert_layer(padded_data)
pooled_output = inputs["pooled_output"]
sequence_output = inputs["sequence_output"]
"""

In [ ]:
# Simpan hasil embedding ke file
with open("path/to/embeddings.txt", "w") as f:
    for i in range(len(data)):
        f.write(data[i] + ": " + str(pooled_output[i].numpy()) + "\n")

In [ ]:
###
!pip install -U tensorflow
!pip install -U tensorflow-text

In [ ]:
###
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
import pandas as pd
import numpy as np

# Load pre-trained BERT model
module_url = "https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-256_A-4/2"
bert_layer = hub.KerasLayer(module_url, trainable=True)
                            #,signature="tokens",
                            #signature_outputs_as_dict=True)

# Load data
#data = pd.read_csv("path/to/data.csv")

# Tokenize data menggunakan pre-trained BERT tokenizer
tokenizer = text.BertTokenizer()
tokenized_data = [tokenizer.tokenize(text) for text in df_train["description"]]

# Padding tokenized data
max_seq_length = 128
padded_data = [tokenizer.convert_tokens_to_ids(tokens) for tokens in tokenized_data]
padded_data = tf.keras.preprocessing.sequence.pad_sequences(padded_data, maxlen=max_seq_length, padding="post")
padded_data = np.array(padded_data)

# Eksekusi pemrosesan data menggunakan model BERT
inputs = bert_layer(padded_data)
pooled_output = inputs["pooled_output"]
sequence_output = inputs["sequence_output"]

In [ ]:
###
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text

text_input = tf.keras.layers.Input(shape=(), dtype=tf.string)
preprocessor = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
encoder_inputs = preprocessor(text_input)
encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-256_A-4/2",
    trainable=True)
outputs = encoder(encoder_inputs)
pooled_output = outputs["pooled_output"]      # [batch_size, 256].
sequence_output = outputs["sequence_output"]  # [batch_size, seq_length, 256].

In [ ]:
embedding_model = tf.keras.Model(text_input, pooled_output)
sentences = tf.constant(["(your text here)"])
print(embedding_model(sentences))

tf.Tensor(
[[-9.17917788e-01  6.18149489e-02 -9.71395373e-01 -5.23985513e-02
   8.16321447e-02  9.86576319e-01 -2.01899260e-01 -5.23172915e-02
  -7.79245496e-02  9.92033064e-01  1.42676950e-01  9.99934554e-01
  -9.99963284e-01 -7.63995230e-01 -3.88732612e-01 -8.59841332e-02
   9.98199046e-01 -9.59068835e-01  9.93820190e-01 -9.31118369e-01
  -9.99998689e-01  9.47735310e-01  9.89432991e-01 -8.93505931e-01
   9.97264802e-01  9.92953300e-01 -1.91433188e-02  3.69281113e-01
   9.95561779e-01 -9.29898024e-02 -8.48858297e-01 -9.79832187e-03
   9.81405020e-01 -9.94961321e-01 -9.96852517e-01 -9.36402142e-01
  -3.63175929e-01  9.89521682e-01 -7.05714881e-01  3.38691950e-01
   3.14439833e-01 -9.72735703e-01 -1.86185479e-01  9.94006991e-01
   9.99933183e-01  3.85377079e-01 -1.20224088e-01  9.92299795e-01
   6.77246451e-02 -1.55153260e-01 -5.58857262e-01 -3.38129878e-01
   1.11717895e-01  9.99935567e-01 -9.84202445e-01 -8.24653804e-01
   8.27391326e-01  9.61776435e-01 -1.24310493e-01 -9.89062786e-01